# Validation Asset Values

The purpose of this notebook is to check the validation asset values and see if they can be recreated using Earth Engine assets.  The albedo, land surface temperature (LST), and NDVI were computed using pre-collection and collection 1 images available in GEE for the Landsat 8 image "LC08_028031_20140708".  Values are being extracted at the AmeriFlux locations used in the other validation work and in the tests.

The values of the validation images do not match up exactly with the TOA computed values, although they are similar (within ~5%).  Based on the comparison with the SR values, the likely explanation for this discrepancy is that the validation assets were generated using LEDAPS at-surface reflectance (not TOA reflectance) and the LST is actually an LST value (and not a brightness temperature like the raw B10 values).

In [1]:
import ee

# This should work assuming "eedisalexi" is in the PYTHONPATH
# import eedisalexi.eedisalexi as eedisalexi
# import eedisalexi.landsat as landsat
# import eedisalexi.utils as utils

# This seems super hacky and is not PEP8 but works for now
import os, sys
module_path = os.path.join(os.path.dirname(os.getcwd()), 'eedisalexi')
sys.path.insert(0, module_path)
import eedisalexi
import landsat
import utils

ee.Initialize()

In [2]:
# AmeriFlux sites adjusted to nearest Landsat cell centroid
# http://sites.ameriflux.lbl.gov/US-NE1/
ne1_xy = [-96.47672812080845, 41.16506126041818]
ne2_xy = [-96.46994024736414, 41.16491226772292]
ne3_xy = [-96.43968912903934, 41.17964494123755]
ne1_geom = ee.Geometry.Point(ne1_xy)
ne2_geom = ee.Geometry.Point(ne2_xy)
ne3_geom = ee.Geometry.Point(ne3_xy)

### Extract the validation asset values at the test point

In [3]:
asset_ws = 'users/cgmorton/disalexi/LC08_028031_20140708/'
asset_albedo_img = ee.Image(asset_ws + 'albedo')
asset_lst_img = ee.Image(asset_ws + 'lst').add(0.01)  # To match IDL codes that use 273.16 to convert K to C
asset_ndvi_img = ee.Image(asset_ws + 'ndvi')

asset_albedo = utils.image_value(asset_albedo_img, xy=ne1_xy)['albedo']
print('albedo: {}'.format(asset_albedo))

asset_lst = utils.image_value(asset_lst_img, xy=ne1_xy)['lst']
print('LST: {}'.format(asset_lst))

asset_ndvi = utils.image_value(asset_ndvi_img, xy=ne1_xy)['ndvi']
print('NDVI: {}'.format(asset_ndvi))

albedo: 0.1990811824798584
LST: 302.4907495117187
NDVI: 0.8460000157356262


### Extract the radiance values from the "raw" pre/C1 images

Images were downloaded from the GCS buckets.  Not sure if it is worth downloading the images from the USGS or amazon also.

https://console.cloud.google.com/storage/browser/gcp-public-data-landsat/LC08/01/028/031/LC08_L1TP_028031_20140708_20170304_01_T1

https://console.cloud.google.com/storage/browser/gcp-public-data-landsat/LC08/PRE/028/031/LC80280312014189LGN00/

In [4]:
# pre = {'B2': 10483, 'B3': 10104, 'B4': 8777, 'B5': 32426,
#        'B6': 15784, 'B7': 10639, 'B10': 23622, 'BQA': 20480}
# c1 = {'B2': 10484, 'B3': 10102, 'B4': 8775, 'B5': 32418,
#       'B6': 15780, 'B7': 10638, 'B10': 23912, 'BQA': 2720}

### Check that the GEE images have the same radiance values

In [5]:
pre_rad_img = ee.Image('LANDSAT/LC8_L1T/LC80280312014189LGN00')
c1_rad_img = ee.Image('LANDSAT/LC08/C01/T1_RT/LC08_028031_20140708')

pre_rad_values = utils.image_value(pre_rad_img, xy=ne1_xy)
print('Red: {}'.format(pre_rad_values['B4']))
print('NIR: {}'.format(pre_rad_values['B5']))
print('LST: {}'.format(pre_rad_values['B10']))
c1_rad_values = utils.image_value(c1_rad_img, xy=ne1_xy)
print('\nRed: {}'.format(c1_rad_values['B4']))
print('NIR: {}'.format(c1_rad_values['B5']))
print('LST: {}'.format(c1_rad_values['B10']))

Red: 7341
NIR: 24527
LST: 26718

Red: 7342
NIR: 24503
LST: 27000


### Print the GEE image TOA values

In [6]:
pre_toa_img = ee.Image('LANDSAT/LC8_L1T_TOA/LC80280312014189LGN00')
c1_toa_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_028031_20140708')

pre_toa_values = utils.image_value(pre_toa_img, xy=ne1_xy)
print('Red: {}'.format(pre_toa_values['B4']))
print('NIR: {}'.format(pre_toa_values['B5']))
print('LST: {}'.format(pre_toa_values['B10']))
c1_toa_values = utils.image_value(c1_toa_img, xy=ne1_xy)
print('\nRed: {}'.format(c1_toa_values['B4']))
print('NIR: {}'.format(c1_toa_values['B5']))
print('LST: {}'.format(c1_toa_values['B10']))

Red: 0.052059274166822433
NIR: 0.4342424273490906
LST: 295.95098876953125

Red: 0.05213770270347595
NIR: 0.43417665362358093
LST: 296.6331787109375


### Print the GEE image SR values

In [7]:
pre_sr_img = ee.Image('LANDSAT/LC8_SR/LC80280312014189').multiply(0.0001)
pre_sr_values = utils.image_value(pre_sr_img, xy=ne1_xy)
print('Red: {}'.format(pre_sr_values['B4']))
print('NIR: {}'.format(pre_sr_values['B5']))
# There is no thermal band in the old GEE Landsat SR images
# print('LST: {}'.format(pre_sr_values['B10']))

Red: 0.0302
NIR: 0.4344


### Compute NDVI

In [8]:
def ndvi(a, b):
    return (float(a) - b) / (a + b)

In [9]:
print('PRE TOA NDVI: {}'.format(ndvi(pre_toa_values['B5'], pre_toa_values['B4'])))
print('C1 TOA NDVI:  {}'.format(ndvi(c1_toa_values['B5'], c1_toa_values['B4'])))
print('PRE SR NDVI:  {}'.format(ndvi(pre_sr_values['B5'], pre_sr_values['B4'])))

PRE TOA NDVI: 0.7858972156398308
C1 TOA NDVI:  0.7855802444441421
PRE SR NDVI:  0.8699956952216961


### Compute albedo

In [10]:
coefs = {'B2': 0.356, 'B4': 0.130, 'B5': 0.373, 'B6': 0.085, 'B7': 0.072}
print('PRE TOA Albedo: {}'.format(sum([c * pre_toa_values[b] for b, c in coefs.items()]) - 0.0018))
print('C1 TOA Albedo:  {}'.format(sum([c * c1_toa_values[b] for b, c in coefs.items()]) - 0.0018))
print('PRE SR Albedo:  {}'.format(sum([c * pre_sr_values[b] for b, c in coefs.items()]) - 0.0018))

PRE TOA Albedo: 0.22201117428094153
C1 TOA Albedo:  0.2220337672501803
PRE SR Albedo:  0.1952039


### Check that the Landsat class is properly renaming the bands

In [11]:
input_image = landsat.Landsat(c1_toa_img).input_image
input_values = utils.image_value(input_image, xy=ne1_xy)
print('\nRed: {}'.format(input_values['red']))
print('NIR: {}'.format(input_values['nir']))
print('LST: {}'.format(input_values['lst']))


Red: 0.05213770270347595
NIR: 0.43417665362358093
LST: 296.6331787109375


### Check the Landsat class NDVI calculation

In [12]:
prep_img = ee.Image(landsat.Landsat(c1_toa_img).prep())
prep_values = utils.image_value(prep_img, xy=ne1_xy)
print('NDVI: {}'.format(prep_values['ndvi']))

NDVI: 0.7855802178382874


### Check the Landsat class albedo calculation

In [13]:
prep_img = ee.Image(landsat.Landsat(c1_toa_img).prep())
prep_values = utils.image_value(prep_img, xy=ne1_xy)
print('Albedo: {}'.format(prep_values['albedo']))

Albedo: 0.2220337672501803


### Check the Landsat class LST calculation

In [14]:
prep_img = ee.Image(landsat.Landsat(c1_toa_img).prep())
prep_values = utils.image_value(prep_img, xy=ne1_xy)
print('LST: {}'.format(prep_values['lst']))

LST: 300.3789960858447
